In [1]:
import numpy as np
import pandas as pd
import datetime as DT

In [7]:
# get the rouped energy data

df = pd.read_csv("C:\\Users\\Hrafnkell\\Documents\\GitHub\\Taming-the-Peak\\Final_Data\\grouped_nrg_winter1314.csv",
                               parse_dates=[0], infer_datetime_format=True)
#df = pd.read_csv("~/Desktop/CUSP/Applied_Data_Science/Taming-the-Peak/Final_Data/grouped_nrg_winter1314.csv",
#                               parse_dates=[0], infer_datetime_format=True)

df.head()

,DateTime,Acorn_grouped,Acorn,stdorToU,KWH/hh (per half hour),LCLid
0,2013-12-01 00:30:00,ACORN-,ACORN-,ToU,0.196000,1
1,2013-12-01 00:30:00,ACORN-U,ACORN-U,Std,0.208056,36
2,2013-12-01 00:30:00,ACORN-U,ACORN-U,ToU,0.165250,8
3,2013-12-01 00:30:00,Adversity,ACORN-K,Std,0.205727,132
4,2013-12-01 00:30:00,Adversity,ACORN-K,ToU,0.175150,20


In [8]:
len(df.DateTime.unique())

4272

In [9]:
df.rename(columns={"KWH/hh (per half hour) ": "nrg_cons", "LCLid": "N"}, inplace=True)
df.head(20)

,DateTime,Acorn_grouped,Acorn,stdorToU,nrg_cons,N
0,2013-12-01 00:30:00,ACORN-,ACORN-,ToU,0.196000,1
1,2013-12-01 00:30:00,ACORN-U,ACORN-U,Std,0.208056,36
2,2013-12-01 00:30:00,ACORN-U,ACORN-U,ToU,0.165250,8
3,2013-12-01 00:30:00,Adversity,ACORN-K,Std,0.205727,132
4,2013-12-01 00:30:00,Adversity,ACORN-K,ToU,0.175150,20
5,2013-12-01 00:30:00,Adversity,ACORN-L,Std,0.179940,251
6,2013-12-01 00:30:00,Adversity,ACORN-L,ToU,0.160944,72
7,2013-12-01 00:30:00,Adversity,ACORN-M,Std,0.163674,89
8,2013-12-01 00:30:00,Adversity,ACORN-M,ToU,0.239250,16
9,2013-12-01 00:30:00,Adversity,ACORN-N,Std,0.132933,120


In [10]:
(df.iloc[3:17,4]*df.iloc[3:17,5]).sum()/df.iloc[3:17,5].sum()

0.18109501181706594

In [20]:
# Spliting Date and time

# temp = pd.DatetimeIndex(grouped_nrg_1314['DateTime'])
# grouped_nrg_1314['Date'] = temp.date
# grouped_nrg_1314['Time'] = temp.time
# del grouped_nrg_1314['DateTime']

In [42]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=df.loc[x.index, "N"])

# Define a dictionary with the functions to apply for a given column:
f = {'nrg_cons': wm}

# Groupby and aggregate with your dictionary:
df_grouped = df.groupby(["DateTime", "Acorn_grouped"]).agg(f)

df_grouped.head()

#grouped_nrg_1314.groupby('Acorn_grouped').aggregate?


nrg_cons
DateTime            Acorn_grouped          
2013-12-01 00:30:00 ACORN-         0.196000
                    ACORN-U        0.200273
                    Adversity      0.181095
                    Affluent       0.255295
                    Comfortable    0.207193

In [62]:
df_grouped.unstack(1).columns[1]

('nrg_cons', 'ACORN-U')

In [63]:
nrg_1314_regress = df_grouped.unstack(1).drop([('nrg_cons', 'ACORN-'),('nrg_cons', 'ACORN-U')], axis = 1)
nrg_1314_regress.head()

nrg_cons                      
Acorn_grouped       Adversity  Affluent Comfortable
DateTime                                           
2013-12-01 00:30:00  0.181095  0.255295    0.207193
2013-12-01 01:00:00  0.163202  0.223506    0.175999
2013-12-01 01:30:00  0.145878  0.195605    0.157086
2013-12-01 02:00:00  0.132853  0.181479    0.140257
2013-12-01 02:30:00  0.120700  0.174701    0.132327

In [64]:
nrg_1314_regress.describe()

nrg_cons                          
Acorn_grouped    Adversity     Affluent  Comfortable
count          4272.000000  4272.000000  4272.000000
mean              0.197358     0.277675     0.236946
std               0.063896     0.089403     0.083664
min               0.092514     0.132146     0.101927
25%               0.148989     0.204205     0.172290
50%               0.188485     0.272868     0.230884
75%               0.247265     0.337459     0.297978
max               0.334082     0.495457     0.417825

In [67]:
output = "~/Desktop/CUSP/Applied_Data_Science/Taming-the-Peak/Final_Data/nrg_winter1314_ready_for_regress.csv"

nrg_1314_regress.to_csv(output)